<a href="https://colab.research.google.com/github/antonpolishko/task-ties/blob/master/CoronaWhyTopicModel_Bio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Common API

In [0]:
import string
import numpy as np
import pandas as pd

#Connect to Google Drive

In [0]:
from pathlib import Path, PurePath
from google.colab import drive

In [0]:
# set data paths, this requires local drive to have a folder calld "COVID-19" with the metadata.csv file
# returns a string to the local path setup
def setup_local_data():
  
  drive.mount('/content/drive')
  drive_path=PurePath('/content/drive/My Drive')
  input_dir = drive_path/'COVID-19'
  print(list(Path(input_dir).glob('*')))
  return input_dir

In [0]:
setup_local_data()

#Unzip JSON

In [0]:
def unzip():
    from zipfile import ZipFile
    
    zf = ZipFile('/content/drive/My Drive/COVID-19/TopicModel/covid_data_full_v5.json.zip', 'r')
    zf.extractall('/content/drive/My Drive/COVID-19/')
    zf.close()

In [0]:
#unzip()

In [0]:
def read_json_to_pandas():
   
    pd_json=pd.read_json (r'/content/drive/My Drive/COVID-19/TopicModel/covid_data_full_v5.json')
    return pd_json

In [0]:
All_json_docs=read_json_to_pandas()
All_json_docs

,_id,title,abstract,text
0,86a998617c077f4fe2ab26214995a3548fbc0fc5,Middle East Respiratory Syndrome and Severe Ac...,The recent emergence of the Middle East respir...,While most CoVs cause the common cold in human...
1,948aaeb2e0be11ad90562bf10d462531a1f00eac,"Integrated, Multi-cohort Analysis Identifies C...",Graphical Abstract Highlights d MVS is a commo...,Clinically relevant respiratory viral signatur...
2,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,Evolutionary Medicine IV. Evolution and Emerge...,The evolutionary history of humans is characte...,The evolutionary history of humans is characte...
3,6599ebbef3d868afac9daa4f80fa075675cf03bc,International aviation emissions to 2025: Can ...,"International aviation is growing rapidly, res...","Sixty years ago, civil aviation was an infant ..."
4,eb5c7f3ff921ad6469b79cc8a3c122648204ece4,2 Mechanisms of diarrhoea,Acute infections of the gastrointestinal tract...,Acute infections of the gastrointestinal tract...
...,...,...,...,...
33370,228650bc0429064d800d4b9c5fb0e00c2533a579,Lipidome profiles of postnatal day 2 vaginal s...,We hypothesized that postnatal development of ...,Early nutritional environment affects long ter...
33371,2246e28681bde69c65dc9081df367bb661997f19,"Secondary Syphilis in Cali, Colombia: New Conc...","Venereal syphilis is a multi-stage, sexually t...",Syphilis is a sexually transmitted disease (ST...
33372,577c6a13f9ef70e9756890fc66e98f537c01ac0a,Replication and shedding of MERS- CoV in Jamai...,The emergence of Middle East respiratory syndr...,Scientific RepoRts | 6:21878 | DOI: 10 .1038/s...
33373,c5c2bc7a07670d6fb970d84a59aab3832752a3f1,Role of the ERK1/2 Signaling Pathway in the Re...,We have previously shown that the infection of...,Arenaviruses are enveloped RNA viruses contain...


In [0]:
All_json_docs['text'][33372]

"Scientific RepoRts | 6:21878 | DOI: 10 .1038/srep21878 Arab Emirates and Egypt 23, [28] [29] [30] . MERS-CoV sequences and virus isolates obtained from dromedary camels in Qatar and the Kingdom of Saudi Arabia showed high sequence identity with those obtained from epidemiologically linked human cases 28, 31 . Together, these data suggest that rather than direct zoonotic transmission from a bat reservoir, dromedary camels are involved as the primary reservoir host for MERS-CoV. Phylogenetic and epidemiological data suggest that rather than a single introduction in the human population, MERS-CoV appears to continue to be transmitted by multiple independent spillover events from dromedary camels 32, 33 . After the emergence of severe acute respiratory syndrome coronavirus (SARS-CoV) in 2003, the targeted focus on reservoir studies in bats has resulted in a vast increase of our knowledge on the genetic diversity of bat coronaviruses. Despite the increase in genetic data on coronavirus div

In [0]:
All_json_docs['abstract'][33372]

'The emergence of Middle East respiratory syndrome coronavirus (MERS-CoV) highlights the zoonotic potential of Betacoronaviruses. Investigations into the origin of MERS-CoV have focused on two potential reservoirs: bats and camels. Here, we investigated the role of bats as a potential reservoir for MERS-CoV. In vitro, the MERS-CoV spike glycoprotein interacted with Jamaican fruit bat (Artibeus jamaicensis) dipeptidyl peptidase 4 (DPP4) receptor and MERS-CoV replicated efficiently in Jamaican fruit bat cells, suggesting there is no restriction at the receptor or cellular level for MERS-CoV. To shed light on the intrinsic host-virus relationship, we inoculated 10 Jamaican fruit bats with MERS-CoV. Although all bats showed evidence of infection, none of the bats showed clinical signs of disease. Virus shedding was detected in the respiratory and intestinal tract for up to 9 days. MERS-CoV replicated transiently in the respiratory and, to a lesser extent, the intestinal tracts and internal

In [0]:
def count_words(sent):
 
   num_words = len(sent.split()) 
   return num_words

In [0]:
count_words(All_json_docs['abstract'][33372])

200

In [0]:
count_words(All_json_docs['text'][33372])

2832

#Preprocessing and cleaned

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def convert_lower_case(data):
    import numpy as np
    return np.char.lower(data)

In [0]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [0]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [0]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [0]:
def remove_numbers(data):
    from string import digits
    remove_digits = str.maketrans('', '', digits)
    res = data.translate(remove_digits)
    return res

In [0]:
def remove_symboles(data):
    import re
    regex = re.compile('[^ s\ a-zA-Z]')#,'[\t\n\r\f\v]')
    data_no_sympoles=regex.sub('',str(data))
    #print(type(data_no_sympoles))
    return data_no_sympoles

In [0]:
remove_symboles(All_json_docs['text'][33372])

'Scientific RepoRts    DOI  srep Arab Emirates and Egypt      MERSCoV sequences and virus isolates obtained from dromedary camels in Qatar and the Kingdom of Saudi Arabia showed high sequence identity with those obtained from epidemiologically linked human cases    Together these data suggest that rather than direct zoonotic transmission from a bat reservoir dromedary camels are involved as the primary reservoir host for MERSCoV Phylogenetic and epidemiological data suggest that rather than a single introduction in the human population MERSCoV appears to continue to be transmitted by multiple independent spillover events from dromedary camels    After the emergence of severe acute respiratory syndrome coronavirus SARSCoV in  the targeted focus on reservoir studies in bats has resulted in a vast increase of our knowledge on the genetic diversity of bat coronaviruses Despite the increase in genetic data on coronavirus diversity in their natural reservoirs only very limited data are avail

In [0]:
def preprocess(data):
    
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = remove_numbers(data)
    data = remove_punctuation(data)
    data = remove_symboles(data)
   
    
    return data

In [0]:
dete_cleaned=preprocess(All_json_docs['text'][33372])

In [0]:
dete_cleaned

' scientific reports doi srep arab emirates egypt mers cov sequences virus isolates obtained dromedary camels qatar kingdom saudi arabia showed high sequence identity obtained epidemiologically linked human cases together data suggest rather direct zoonotic transmission bat reservoir dromedary camels involved primary reservoir host mers cov phylogenetic epidemiological data suggest rather single introduction human population mers cov appears continue transmitted multiple independent spillover events dromedary camels emergence severe acute respiratory syndrome coronavirus sars cov targeted focus reservoir studies bats resulted vast increase knowledge genetic diversity bat coronaviruses despite increase genetic data coronavirus diversity natural reservoirs limited data available impact viruses reservoir host controlled infection experiments coronaviruses reservoir hosts performed understand drivers mers cov emergence comprehensive understanding interaction virus natural intermediate rese

In [0]:
#len(dete_cleaned)
#dete_cleaned.size

In [0]:
def cleaned_COVID19(All_json_docs):
    All_json_docs['cleaned_text'] = All_json_docs['text'].apply(preprocess)
    covid_dataset_dropnan=All_json_docs.dropna()
    All_json_docs.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/covid_data_full_v5_cleaned.csv',index=False,sep=',',header=True)
    covid_dataset_dropnan.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/covid_data_full_v5_cleaned_dropnan.csv',index=False,sep=',',header=True)
    return All_json_docs

In [0]:
cleaned_COVID19(All_json_docs)

#Topic Modelling

##Subset

In [0]:
def read_covid_dataset(columns,n):
    data = pd.read_csv("/content/drive/My Drive/COVID-19/TopicModel/covid_data_full_v5_cleaned_dropnan.csv",usecols= columns, nrows=n,index_col=0)#usecols= ['_id','text','cleaned_text']
    return data


In [0]:
columns=['_id','cleaned_text']
covid_dataset=read_covid_dataset(columns,10)

covid_dataset.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/covid_data_full_v5_cleaned_dropnan_10.csv',index=False,sep=',',header=True)

In [0]:
covid_dataset

,cleaned_text
_id,
86a998617c077f4fe2ab26214995a3548fbc0fc5,covs cause common cold humans infection two r...
948aaeb2e0be11ad90562bf10d462531a1f00eac,clinically relevant respiratory viral signatu...
ab680d5dbc4f51252da3473109a7885dd6b5eb6f,evolutionary history humans characterized dyn...
6599ebbef3d868afac9daa4f80fa075675cf03bc,sixty years ago civil aviation infant industr...
eb5c7f3ff921ad6469b79cc8a3c122648204ece4,acute infections gastrointestinal tract far c...
b87b790c96c75faa22a085cb560f7b3d8e018b24,three domains life bacteria archaea eukarya o...
68c0bb1989b6ca2b38da32a0d992027db39f80bc,also noteworthy tang district commander yuen ...
5ad06be75e9c306aa10fea704f93d4ba90623a15,random peptide libraries antigen fragment lib...
032207023bce7ab4550c38283ac7e2964d3c6561,viruses use diverse mechanisms outcompete cel...


In [0]:
def all_words(covid_dataset):
    all_words=[]
    for line in covid_dataset['cleaned_text']:
        wod_line=line.split(' ')
        for word in wod_line:
            all_words.append(word)
    print(len(all_words))
    return all_words




In [0]:
total_words=all_words(covid_dataset)

30166


In [0]:
type(total_words)

list

In [0]:
def unique_words(x):
    pure_list=list(dict.fromkeys(x))
    import csv
    with open('/content/drive/My Drive/COVID-19/TopicModel/all_terms_10.csv',mode='w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_NONE,delimiter=',')
        wr.writerow(pure_list)
    return len(pure_list)

mylist = unique_words(total_words)
mylist

6260

In [0]:
data = pd.read_csv("/content/drive/My Drive/COVID-19/TopicModel/all_terms_10.csv")
data

,Unnamed: 0,covs,cause,common,cold,humans,infection,two,recently,emerged,sars,cov,mers,resulted,severe,pulmonary,disease,alarmingly,high,case,fatality,rates,first,guangdong,province,china,winter,rate,nosocomial,transmission,healthcare,professionals,combined,lack,precedence,outbreak,spread,across,countries,infecting,...,ratios,preparations,crossreactivity,culturenegatives,positives,investigation,syncytia,examples,agreement,pictures,evidently,measurable,evaluation,tests,naturally,occurring,epidemiologic,replaced,peroxidase,microtiter,plates,outpatient,proved,opinion,potency,labeling,monoclonal,desirable,volumes,flow,interpretation,printing,computerized,habermehl,directed,kind,therapy,started,fulfill,requirements


In [0]:
#data[]
s=data[data.columns[1]]
type(s)
s.name

'covs'

In [0]:
data.where(df =='a')

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,Unnamed: 0,covs,cause,common,cold,humans,infection,two,recently,emerged,sars,cov,mers,resulted,severe,pulmonary,disease,alarmingly,high,case,fatality,rates,first,guangdong,province,china,winter,rate,nosocomial,transmission,healthcare,professionals,combined,lack,precedence,outbreak,spread,across,countries,infecting,...,ratios,preparations,crossreactivity,culturenegatives,positives,investigation,syncytia,examples,agreement,pictures,evidently,measurable,evaluation,tests,naturally,occurring,epidemiologic,replaced,peroxidase,microtiter,plates,outpatient,proved,opinion,potency,labeling,monoclonal,desirable,volumes,flow,interpretation,printing,computerized,habermehl,directed,kind,therapy,started,fulfill,requirements


## TF-IDF

In [0]:
import nltk
nltk.download('punkt')
def tokenizer(text):
    tokens = nltk.word_tokenize(text)
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def tfidf():
    from sklearn.feature_extraction.text import TfidfVectorizer
    v = TfidfVectorizer(tokenizer=tokenizer,analyzer ='word')
    x = v.fit_transform(covid_dataset.cleaned_text.values)
    data=x.toarray()
    headers=v.get_feature_names() 
    df = pd.DataFrame (data, columns =headers,index=covid_dataset.index.values)
    df.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/covid_TFIDF_10.csv')#,index=True,sep=',',header=True)
    return df

In [0]:
covid_dataset.index.values

array(['86a998617c077f4fe2ab26214995a3548fbc0fc5',
       '948aaeb2e0be11ad90562bf10d462531a1f00eac',
       'ab680d5dbc4f51252da3473109a7885dd6b5eb6f',
       '6599ebbef3d868afac9daa4f80fa075675cf03bc',
       'eb5c7f3ff921ad6469b79cc8a3c122648204ece4',
       'b87b790c96c75faa22a085cb560f7b3d8e018b24',
       '68c0bb1989b6ca2b38da32a0d992027db39f80bc',
       '5ad06be75e9c306aa10fea704f93d4ba90623a15',
       '032207023bce7ab4550c38283ac7e2964d3c6561',
       '6c9a692eb00e9563f550ab57838c7b29d6731e54'], dtype=object)

In [0]:
tfidf()
   

,a,ab,abatement,abbreviations,ability,able,abnormal,abnormalities,abolish,abolished,abolishes,abortively,abroad,abrogation,abs,absence,absolute,absolved,absorbed,absorption,absorptive,abundance,abundant,abuse,abuses,acceptance,accepted,access,accessibility,accessible,accessory,accommodated,accommodates,accommodation,accompanied,accompanies,accompany,accordance,according,account,...,xinhua,xis,xm,yamashiro,yb,year,years,yeast,yellow,yersinia,yet,yeung,yield,yielded,yn,yolken,yoph,yops,young,youth,youtu,youtube,yr,yu,yuen,zaas,zbp,zero,zhang,zhejiang,zheng,zhu,zip,zone,zonula,zot,zuoer,zwick,zygote,zytica
86a998617c077f4fe2ab26214995a3548fbc0fc5,0.004553,0.000000,0.000000,0.000000,0.000000,0.006036,0.00000,0.00000,0.00000,0.00000,0.00000,0.011243,0.000000,0.000000,0.000000,0.018107,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.044972,0.000000,0.000000,0.000000,0.008362,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.033381,0.000000,0.000000,0.000000,0.006036,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.014868,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
948aaeb2e0be11ad90562bf10d462531a1f00eac,0.019171,0.000000,0.000000,0.000000,0.006923,0.020331,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.002541,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.003130,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.011245,0.000000,0.000000,0.000000,0.002541,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.009391,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018936,0.004734,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
ab680d5dbc4f51252da3473109a7885dd6b5eb6f,0.005119,0.010746,0.000000,0.000000,0.006162,0.013572,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.009401,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.008358,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.015013,0.000000,0.009401,0.000000,0.000000,0.000000,0.000000,0.000000,0.010746,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
6599ebbef3d868afac9daa4f80fa075675cf03bc,0.013104,0.000000,0.023113,0.000000,0.000000,0.004963,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.002482,0.004623,0.000000,0.00000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004623,0.000000,0.000000,0.003438,0.00000,0.00000,0.007859,0.000000,0.018340,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.030195,0.016470,0.000000,0.000000,0.000000,0.002482,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
eb5c7f3ff921ad6469b79cc8a3c122648204ece4,0.013444,0.000000,0.000000,0.000000,0.016183,0.000000,0.00664,0.00664,0.00664,0.01328,0.00664,0.000000,0.000000,0.000000,0.000000,0.003564,0.000000,0.000000,0.01328,0.073038,0.00664,0.00664,0.000000,0.000000,0.000000,0.00664,0.000000,0.004938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004938,0.00664,0.00664,0.000000,0.000000,0.000000,...

In [0]:
df['corona']

0    0.000000
1    0.000000
2    0.012641
3    0.000000
4    0.000000
5    0.000000
6    0.000000
7    0.000000
8    0.000000
9    0.000000
Name: corona, dtype: float64

#**CorelationCoeffitien matrix (H)**
https://towardsdatascience.com/word-embedding-using-bert-in-python-dd5a86c00342

##FastText
https://fasttext.cc/docs/en/english-vectors.html

https://medium.com/@h_bushroh/text-similarity-with-fasttext-word-embeddings-c765d97df682
https://github.com/facebookresearch/fastText/blob/master/docs/pretrained-vectors.md

https://github.com/facebookresearch/fastText/blob/master/docs/pretrained-vectors.md

https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip

https://github.com/ncbi-nlp/BioSentVec


In [0]:
import os
os.chdir('/content/drive/My Drive/COVID-19/TopicModel/')

In [0]:
!ls

 all_terms_10.csv
'Copy of CoronaWhy.ipynb'
 CoronaWhy.ipynb
 covid_data_full_v5_cleaned.csv
 covid_data_full_v5_cleaned_dropnan_10.csv
 covid_data_full_v5_cleaned_dropnan.csv
 covid_data_full_v5.json
 covid_data_full_v5.json.zip
 covid_TFIDF_10.csv
 __MACOSX
 old
 spark-3.0.0-preview2-bin-hadoop2.7
 spark-3.0.0-preview2-bin-hadoop2.7.tgz
 wiki.en.zip


###Download

In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip

--2020-04-04 21:48:39--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  12.4MB/s    in 13m 30s 

2020-04-04 22:02:11 (12.2 MB/s) - ‘wiki.en.zip’ saved [10356881291/10356881291]



In [0]:
!unzip '/content/drive/My Drive/COVID-19/TopicModel/wiki.en.zip'

Archive:  /content/drive/My Drive/COVID-19/TopicModel/wiki.en.zip
  inflating: wiki.en.vec             
  inflating: wiki.en.bin             


###Load Model

In [0]:
#https://radimrehurek.com/gensim/models/deprecated/keyedvectors.html
#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Word2Vec_FastText_Comparison.ipynb
#from gensim.models.fasttext import FastText
from gensim.models import Word2Vec, KeyedVectors
fast_text_vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/COVID-19/TopicModel/wiki.en.vec')
#fast_text_model=FastText.load_fasttext_format('/content/drive/My Drive/COVID-19/TopicModel/wiki.en.vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
fast_text_vec.most_similar('zot')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('אות', 0.5454506278038025),
 ('zoth', 0.5341421365737915),
 ('שמחה', 0.5249664783477783),
 ('איר', 0.5166455507278442),
 ('shelom', 0.514742374420166),
 ('zom', 0.5145315527915955),
 ('קיר', 0.5143148899078369),
 ('zotl', 0.5142924189567566),
 ('varim', 0.5141097903251648),
 ('אומות', 0.511289119720459)]

In [0]:
fast_text_vec.cosine_similarities(fast_text_vec.get_vector('dog'),[fast_text_vec.get_vector('cat'),fast_text_vec.get_vector('bit')])

array([0.6380517 , 0.20742023], dtype=float32)

In [0]:
data = pd.read_csv("/content/drive/My Drive/COVID-19/TopicModel/all_terms_10.csv")
data

In [0]:
all_words=data.columns[1:]
len(all_words)
all_words

Index(['covs', 'cause', 'common', 'cold', 'humans', 'infection', 'two',
       'recently', 'emerged', 'sars',
       ...
       'interpretation', 'printing', 'computerized', 'habermehl', 'directed',
       'kind', 'therapy', 'started', 'fulfill', 'requirements'],
      dtype='object', length=6259)

In [0]:
all_fast_text_vec_list=[]
for w in all_words:
    print(type(w))
    all_fast_text_vec_list.append(fast_text_vec.get_vector(w))

#all_fast_text_vec_list

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

KeyError: ignored

In [0]:
for w in all_words:
    

In [0]:
s=data[data.columns[1]]
type(s)
s.name

##Bio word2vec
https://github.com/ncbi-nlp/BioSentVec
https://github.com/jakerochlinmarcus/biomedical-word-embeddings

###Download

In [0]:
import os
os.chdir('/content/drive/My Drive/COVID-19/TopicModel/')

In [0]:
!ls

 all_terms_10.csv
'Copy of CoronaWhy.ipynb'
 CoronaWhy.ipynb
 covid_data_full_v5_cleaned.csv
 covid_data_full_v5_cleaned_dropnan_10.csv
 covid_data_full_v5_cleaned_dropnan.csv
 covid_data_full_v5.json
 covid_data_full_v5.json.zip
 covid_TFIDF_10.csv
 __MACOSX
 old
 spark-3.0.0-preview2-bin-hadoop2.7
 spark-3.0.0-preview2-bin-hadoop2.7.tgz
 wiki.en.vec


In [0]:
!wget https://ftp.ncbi.nlm.nih.gov/pub/lu/Suppl/BioSentVec/BioWordVec_PubMed_MIMICIII_d200.vec.bin

--2020-04-05 12:10:53--  https://ftp.ncbi.nlm.nih.gov/pub/lu/Suppl/BioSentVec/BioWordVec_PubMed_MIMICIII_d200.vec.bin
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 2607:f220:41e:250::11
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13451441787 (13G) [application/octet-stream]
Saving to: ‘BioWordVec_PubMed_MIMICIII_d200.vec.bin’

BioWordVec_PubMed_M 100%[===================>]  12.53G  41.6MB/s    in 5m 15s  

2020-04-05 12:16:15 (40.7 MB/s) - ‘BioWordVec_PubMed_MIMICIII_d200.vec.bin’ saved [13451441787/13451441787]



In [0]:
!wget http://evexdb.org/pmresources/vec-space-models/PubMed-and-PMC-w2v.bin

--2020-04-05 12:48:11--  http://evexdb.org/pmresources/vec-space-models/PubMed-and-PMC-w2v.bin
Resolving evexdb.org (evexdb.org)... 130.232.253.44
Connecting to evexdb.org (evexdb.org)|130.232.253.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3317501420 (3.1G) [application/octet-stream]
Saving to: ‘PubMed-and-PMC-w2v.bin’

PubMed-and-PMC-w2v. 100%[===================>]   3.09G  1.10MB/s    in 52m 57s 

2020-04-05 13:41:09 (1020 KB/s) - ‘PubMed-and-PMC-w2v.bin’ saved [3317501420/3317501420]



###Load

In [0]:
!ls

drive  PubMed-and-PMC-w2v.bin  sample_data


In [0]:
!mv ./PubMed-and-PMC-w2v.bin '/content/drive/My Drive/COVID-19/TopicModel/'

In [0]:
from gensim.models import Word2Vec, KeyedVectors
Bio_text_vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/COVID-19/TopicModel/PubMed-and-PMC-w2v.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
Bio_text_vec.most_similar('zot')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ctxA', 0.8632912039756775),
 ('tcpI', 0.8289449214935303),
 ('tcpA', 0.8145631551742554),
 ('toxR', 0.8023501634597778),
 ('ctxB', 0.7812706232070923),
 ('toxS', 0.7810416221618652),
 ('rstR', 0.7737479209899902),
 ('rtxC', 0.7607611417770386),
 ('ctxAB', 0.7562148571014404),
 ('ompU', 0.7544364333152771)]

In [0]:
def H1():
    data = pd.read_csv("/content/drive/My Drive/COVID-19/TopicModel/all_terms_10.csv")
    all_words=data.columns[1:]
    len(all_words)
    all_words
    all_Bio_text_vec_list=[]
    Bio_text_word=[]
    Not_Bio_text_word=[]
    for w in all_words:
        #print(type(w))
        try:
            all_Bio_text_vec_list.append(Bio_text_vec.get_vector(w))
            Bio_text_word.append(w)
        except:
            Not_Bio_text_word.append(w)
            pass
    print(len(all_Bio_text_vec_list))#5751
    print(len(Bio_text_word))#5751
    print(len(Not_Bio_text_vec_list))#508
    all_Bio_word_cos=[]
    for w in Bio_text_word:
        cos=Bio_text_vec.cosine_similarities(Bio_text_vec.get_vector(w),all_Bio_text_vec_list)
        all_Bio_word_cos.append(cos)
    df_H1 = pd.DataFrame(all_Bio_word_cos, index =Bio_text_word,columns=Bio_text_word)
    df_H1.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/H1_10doc_Bio.csv')
    return df_H1



,Unnamed: 0,covs,cause,common,cold,humans,infection,two,recently,emerged,sars,cov,mers,resulted,severe,pulmonary,disease,alarmingly,high,case,fatality,rates,first,guangdong,province,china,winter,rate,nosocomial,transmission,healthcare,professionals,combined,lack,precedence,outbreak,spread,across,countries,infecting,...,ratios,preparations,crossreactivity,culturenegatives,positives,investigation,syncytia,examples,agreement,pictures,evidently,measurable,evaluation,tests,naturally,occurring,epidemiologic,replaced,peroxidase,microtiter,plates,outpatient,proved,opinion,potency,labeling,monoclonal,desirable,volumes,flow,interpretation,printing,computerized,habermehl,directed,kind,therapy,started,fulfill,requirements


In [0]:
H1()


,cause,common,cold,humans,infection,two,recently,emerged,sars,cov,mers,resulted,severe,pulmonary,disease,alarmingly,high,case,fatality,rates,first,guangdong,province,china,winter,rate,nosocomial,transmission,healthcare,professionals,combined,lack,precedence,outbreak,spread,across,countries,infecting,resulting,staggering,...,sheep,antirabbit,ratios,preparations,crossreactivity,positives,investigation,syncytia,examples,agreement,pictures,evidently,measurable,evaluation,tests,naturally,occurring,epidemiologic,replaced,peroxidase,microtiter,plates,outpatient,proved,opinion,potency,labeling,monoclonal,desirable,volumes,flow,interpretation,printing,computerized,directed,kind,therapy,started,fulfill,requirements
cause,1.000000,0.413175,0.102242,0.325554,0.188100,0.052576,0.058178,0.139699,0.034599,-0.090436,0.048563,0.274290,0.488538,0.134631,0.242175,0.202825,0.056520,0.233239,0.237114,0.269254,0.017792,-0.016543,0.084911,0.033636,0.095117,0.248077,0.217077,0.147095,0.052720,0.037410,0.038690,0.232946,0.038070,0.179389,0.235687,0.075153,0.140509,0.127071,0.385667,0.220113,...,0.077966,-0.027224,0.072355,0.038698,-0.006729,0.053160,0.061519,0.065957,0.101087,-0.111059,0.014782,0.188121,0.176197,0.017995,0.021138,0.128173,0.388050,0.057786,-0.002586,-0.036910,-0.076610,-0.029328,-0.001672,0.095677,0.089152,0.067816,-0.055450,-0.063981,0.100671,0.018222,-0.010615,0.102158,-0.053786,-0.058100,0.002444,0.251904,0.085236,-0.043523,0.201842,0.077913
common,0.413175,1.000000,0.088936,0.281255,0.119688,0.179981,0.176890,0.191849,0.007441,-0.035447,0.039397,0.023641,0.360218,0.066836,0.207187,0.190805,0.111604,0.164547,0.067519,0.070827,0.160728,0.089714,0.099663,0.051445,0.077428,0.043772,0.165288,0.060465,0.083183,0.098303,0.145596,0.093508,0.132256,0.095698,0.189787,0.179792,0.210970,0.219643,0.171983,0.181835,...,0.058310,-0.035518,-0.006555,-0.002601,0.119931,0.081247,0.102581,-0.020025,0.229934,0.056874,0.022947,0.256946,0.109617,-0.008183,-0.006715,0.219067,0.389546,0.127342,0.050737,-0.015222,0.018537,0.014346,0.065726,0.110798,0.128043,-0.015673,0.003967,0.010312,0.291738,-0.058294,0.010011,0.004703,0.008015,-0.011336,0.110761,0.179903,0.024987,-0.028682,0.064279,0.013067
cold,0.102242,0.088936,1.000000,0.054694,0.118312,0.080489,-0.007555,0.017831,-0.003591,-0.099631,-0.030497,0.051632,0.210313,0.040281,-0.045571,-0.061690,0.132541,0.002572,0.060261,0.030404,0.055166,0.031256,0.076195,0.068496,0.353327,0.045849,0.008385,-0.005509,-0.034888,-0.040794,0.125392,0.054788,0.032570,0.101796,0.103806,0.020818,-0.002031,0.005986,0.083443,0.007844,...,0.117648,0.151942,-0.040363,0.087423,-0.023456,-0.000662,0.048131,0.016175,-0.015859,0.006078,0.137421,0.152105,-0.027604,-0.023652,0.103433,0.031767,0.062369,-0.041750,0.125027,0.090761,0.083077,0.177642,0.011296,0.025650,0.053155,-0.032745,0.067526,0.007317,0.013747,0.158488,0.148477,-0.036670,0.055259,-0.050234,-0.061996,0.010221,0.002539,0.119162,-0.027240,0.024765
humans,0.325554,0.281255,0.054694,1.000000,0.342417,0.085788,0.270189,0.187509,0.080167,-0.106411,-0.032038,0.143469,0.251167,0.105259,0.237502,0.178389,0.190601,0.170809,0.097799,0.130528,0.094835,0.024126,0.139420,0.070750,0.121437,0.056948,0.178893,0.330984,0.092816,0.092003,0.078912,0.185461,0.061211,0.228893,0.274211,0.185101,0.262663,0.397432,0.155186,0.180956,...,0.416270,-0.024267,-0.001957,0.175143,0.151187,0.112306,0.160898,0.067922,0.158449,0.182778,0.037326,0.125900,0.106016,0.003006,0.122862,0.384753,0.277300,0.284650,-0.011045,-0.042752,-0.081118,-0.054289,-0.007872,0.101931,0.075834,0.163015,-0.046177,-0.007844,0.179279,-0.024273,-0.031517,0.062076,-0.094293,-0.044740,0.038998,0.069247,0.104439,-0.014203,0.082094,0.073054
infection,0.188100,0.119688,0.118312,0.342417,1.000000,0.081719,0.056076,0.150095,0.096062,-0.039749,-0.005629,0.021726,0.247960,0.196114,0.328072,0.194888,0.154154,0.194083,0.273924,0.177727,0.186894,-0.053863,0.177137,0.004426,0.150167,0.118823,0.483404,0.508091,0.085710,0.041585,0.044971,0.1283

In [0]:
def H2_50():
    data_H1 = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/H1_10doc_Bio.csv',index_col=0)#,header=None)
    for col in data_1.columns:
        data_H1.loc[data_H1[col] < .50, col] = 0
    data_H1.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/H2_10doc_Bio_50.csv')
    return data_H1

In [0]:
H2_50()

,cause,common,cold,humans,infection,two,recently,emerged,sars,cov,mers,resulted,severe,pulmonary,disease,alarmingly,high,case,fatality,rates,first,guangdong,province,china,winter,rate,nosocomial,transmission,healthcare,professionals,combined,lack,precedence,outbreak,spread,across,countries,infecting,resulting,staggering,...,sheep,antirabbit,ratios,preparations,crossreactivity,positives,investigation,syncytia,examples,agreement,pictures,evidently,measurable,evaluation,tests,naturally,occurring,epidemiologic,replaced,peroxidase,microtiter,plates,outpatient,proved,opinion,potency,labeling,monoclonal,desirable,volumes,flow,interpretation,printing,computerized,directed,kind,therapy,started,fulfill,requirements
cause,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
common,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
cold,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
humans,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
infection,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.508091,0.0,0.0,0.0,0.0,0.0,0.510116,0.505389,0.0,0.0,0.581102,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kind,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000
therapy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.000000
started,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000
fulfill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [0]:
#(data_1 == 0).astype(int).sum(axis=1)

cause           5727
common          5737
cold            5744
humans          5744
infection       5727
                ... 
kind            5747
therapy         5740
started         5736
fulfill         5738
requirements    5736
Length: 5751, dtype: int64

#**Topic Doument**

##Inverse Martix H

In [0]:
v=wHT--->wv/H or w=vHi

In [0]:
def inv_H2():
    data_H2 = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/H2_10doc_Bio_50.csv',index_col=0)
    df_inv = pd.DataFrame(np.linalg.pinv(data_H2.values), data_H2.columns, data_H2.index)
    df_inv.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/H2_Inv_Bio_50.csv')
    return df_inv

In [0]:
inv_H2()

,cause,common,cold,humans,infection,two,recently,emerged,sars,cov,mers,resulted,severe,pulmonary,disease,alarmingly,high,case,fatality,rates,first,guangdong,province,china,winter,rate,nosocomial,transmission,healthcare,professionals,combined,lack,precedence,outbreak,spread,across,countries,infecting,resulting,staggering,...,sheep,antirabbit,ratios,preparations,crossreactivity,positives,investigation,syncytia,examples,agreement,pictures,evidently,measurable,evaluation,tests,naturally,occurring,epidemiologic,replaced,peroxidase,microtiter,plates,outpatient,proved,opinion,potency,labeling,monoclonal,desirable,volumes,flow,interpretation,printing,computerized,directed,kind,therapy,started,fulfill,requirements
cause,-0.288228,0.244739,0.215348,0.381192,0.008387,-0.197696,0.099876,-0.583393,-0.129299,-0.327087,-0.369033,0.621151,-0.267005,-0.107845,0.687629,0.187272,-0.987488,-0.080956,0.387512,0.563130,-0.094813,-0.069651,-0.355274,0.229400,-0.296270,-0.743929,-0.054424,-0.741395,0.354039,0.100693,0.970056,0.050870,-3.930552e-15,0.141560,-0.281334,-0.146262,0.227848,-0.390221,-0.746923,0.206849,...,0.112445,-0.106682,-0.574073,0.071780,0.271646,0.537630,0.319095,0.142677,-0.179118,0.421308,0.481877,0.673825,-0.574316,0.074207,-0.792602,0.067310,0.208622,-0.856065,-0.379329,-0.090740,-0.056862,-0.028304,0.416684,0.382681,-0.206283,0.173654,-0.003020,0.715009,0.162789,0.000000e+00,-0.065205,-0.397661,-0.136779,0.003835,-0.211383,0.384549,-0.418773,0.513717,-0.324794,0.170979
common,0.244740,-0.852152,-0.370075,0.050872,0.203076,-0.120816,0.462354,0.217656,0.198953,-1.225227,0.059932,-0.170541,-0.080816,0.436250,0.027613,0.147445,-0.465024,0.240325,0.286448,0.815934,0.044198,0.051225,0.373604,-0.086141,-0.105064,-0.235716,0.102739,-1.036666,0.037728,0.649337,0.994268,0.122018,-3.841616e-15,0.303534,0.162403,-0.169408,0.068496,-0.183980,-0.233856,-0.483050,...,0.663656,-0.285933,-0.465405,0.227305,0.322901,0.178290,0.290122,-0.012890,0.225261,-0.546626,0.171839,-0.522152,0.326321,-0.105634,-0.354599,-0.081220,-0.923797,-0.811654,0.005376,0.267899,0.104891,0.117874,-0.466492,0.224232,-0.030490,0.255063,-0.607262,-0.094096,0.169687,-5.174403e-20,-0.776634,-0.313750,-0.044532,0.559939,0.069583,-0.195873,0.024911,-0.411687,0.465998,-0.572656
cold,0.215347,-0.370076,3.386426,1.340260,0.956111,-0.809401,-1.235385,1.141186,0.423262,3.099216,0.848266,-2.722945,0.163893,0.780729,-0.945305,-0.886569,6.568812,0.347428,-2.345982,-3.711262,-0.588243,1.226699,1.551563,-1.595781,1.332568,2.505704,-0.022758,1.881140,-1.282033,-1.258697,-7.154403,0.508850,2.586031e-14,0.868545,0.172952,0.032303,-0.603321,2.514045,2.914226,-1.247417,...,-3.453570,1.387929,0.629695,-0.755418,-2.167149,-4.065508,-3.955948,-0.142625,2.067698,-0.622472,-2.769459,-2.084928,1.445897,1.407449,4.195151,-0.448716,-2.359987,-4.683129,0.961715,-0.005109,0.366325,-2.086782,0.843617,0.278711,1.300238,-1.394097,-0.137879,-5.016841,-1.725528,-1.553451e-17,5.065754,3.466696,0.672370,-0.653272,2.284418,-0.959129,0.331371,-0.143944,-0.295807,1.034951
humans,0.381192,0.050872,1.340259,-0.739102,-0.184063,-0.598383,-0.348778,-0.859294,-0.374942,0.750855,1.170904,0.213886,0.217679,-0.091006,0.123898,-0.126936,0.766008,-0.153733,0.054773,-0.820263,-0.166280,0.209113,-0.320872,-0.028536,-0.638035,0.070316,-0.487158,0.245780,-0.557216,-0.275346,-1.443750,0.428246,5.902836e-15,0.627786,-0.047736,1.160055,0.476628,-0.154098,0.404517,-0.171948,...,0.374574,0.031764,0.947629,-0.507566,-0.477423,-0.247428,-0.337261,0.703393,-0.267355,0.460106,-0.284568,1.240659,-0.818205,-0.070631,0.037826,-0.118080,0.999228,2.888879,0.226851,0.115910,-0.300519,0.503060,0.467798,0.172205,0.349746,-0.403542,0.091340,0.307250,-0.036470,6.956705e-18,0.482560,0.709496,-0.086078,0.452329,-0.056860,0.247005,-0.358821,0.355126,-0.704436,0.571659
infection,0.008386,0.203076,0.956110,-0.184062,-0.355209,-0.284717,-0.416737,-0.145459,-0.193165,0.968377,0.824274,-0.209225,0.345126,-0.058051,-0.208938,-0.271635,1.225153,-0.202

In [0]:
def  W():
    inv_H2 = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/H2_Inv_Bio_50.csv',index_col=0)
    inv_H2.sort_index(axis=1, inplace=True)
    inv_H2.sort_index(axis=0, inplace=True)
    tfIdf_bio = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/covid_TFIDF_10.csv',index_col=0)
    tfIdf_bio_2=tfIdf_bio[inv_H2.columns]
    W=tfIdf_bio_2.dot(inv_H2)
    W.index=tfIdf_bio.index
    W.to_csv(r'/content/drive/My Drive/COVID-19/TopicModel/W_Bio_50.csv')
    return W

In [0]:
'''inv_H2 = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/H2_Inv_Bio_50.csv',index_col=0)
inv_H2.sort_index(axis=1, inplace=True)
inv_H2.sort_index(axis=0, inplace=True)
tfIdf_bio = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/covid_TFIDF_10.csv',index_col=0)#,usecols=inv_H2.columns)
tfIdf_bio_2=tfIdf_bio[inv_H2.columns]
tfIdf_bio_2'''

"inv_H2 = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/H2_Inv_Bio_50.csv',index_col=0)\ninv_H2.sort_index(axis=1, inplace=True)\ninv_H2.sort_index(axis=0, inplace=True)\ntfIdf_bio = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/covid_TFIDF_10.csv',index_col=0)#,usecols=inv_H2.columns)\ntfIdf_bio_2=tfIdf_bio[inv_H2.columns]\ntfIdf_bio_2"

In [0]:
#tfIdf_bio_2.dot(inv_H2)

In [0]:
#tfIdf_bio[inv_H2.columns]
W()# Doc-topic() topic is column, Doc is row)

,a,ab,abatement,abbreviations,ability,able,abnormal,abnormalities,abolish,abolished,abolishes,abortively,abroad,abrogation,abs,absence,absolute,absolved,absorbed,absorption,absorptive,abundance,abundant,abuse,abuses,acceptance,accepted,access,accessibility,accessible,accessory,accommodated,accommodates,accommodation,accompanied,accompanies,accompany,accordance,according,account,...,worlds,worldwide,worse,worst,would,wounds,written,wrote,www,xi,xia,xie,xis,xm,yb,year,years,yeast,yellow,yersinia,yet,yield,yielded,yn,yops,young,youth,youtube,yr,yu,zero,zhang,zhejiang,zheng,zhu,zip,zone,zonula,zot,zygote
86a998617c077f4fe2ab26214995a3548fbc0fc5,0.460935,-1.074855e-14,-0.298150,-0.314394,-1.508156,-1.096905,2.111429,4.175764,-1.160194,-0.412763,0.193602,0.703859,-0.812519,-0.180455,-2.104143e-14,1.643565,-2.738175,1.102463,-0.370662,-1.372302,1.697831,1.721103,-3.958260,-0.122738,-1.385939,-0.270165,-1.770498,-0.742053,0.663081,-1.269406,4.497164e-02,-0.975076,1.053534,-1.067793,-3.300038,2.854167,-0.197579,-0.898849,0.444022,1.246996,...,1.828296,0.409952,1.028479,0.310690,4.088937,0.345697,-1.019142,1.915664,-2.168925,-7.393889,-4.570836,2.843601,3.369717,3.809799,0.380573,2.547668,-0.203742,-3.631828,0.546002,2.101826,-0.724527,0.576278,-0.633111,6.446187,3.081228e-14,0.542817,-0.827918,0.807012,2.102257,-3.261064,1.058825,-0.471856,-2.079947,1.165502,-2.454552,-0.642242,-0.618172,0.227445,-0.649165,0.414810
948aaeb2e0be11ad90562bf10d462531a1f00eac,0.337884,-8.513721e-17,-0.036184,0.280703,0.067410,0.625925,-0.645145,0.148022,-0.364650,0.185770,-0.184402,-0.190721,0.306871,1.066101,1.141656e-14,-0.745930,0.850521,0.318068,0.569155,0.092068,-0.156027,-0.666020,0.667546,-0.293164,0.739878,0.456776,1.187053,0.516724,-0.674488,-0.021932,-4.516173e-15,0.523607,-0.179941,0.197487,0.258705,-0.175149,-0.531647,0.659825,-0.491059,-0.793422,...,-0.707331,0.626227,-0.882800,0.905802,-1.426347,0.716577,0.530841,-0.809478,0.846036,1.546432,0.572082,-1.086539,0.040479,-0.965339,0.675438,-0.347800,0.484620,1.396174,-0.639457,-0.540418,-0.109933,-0.317493,-0.260603,-0.503037,-6.750543e-15,0.377659,0.037918,-0.346429,-0.147153,1.348608,-0.710044,1.273608,0.108912,-2.022796,0.877152,0.646405,0.344513,-0.491522,-0.423666,0.034344
ab680d5dbc4f51252da3473109a7885dd6b5eb6f,0.168321,1.074577e-02,0.130569,0.017753,-0.227951,0.070014,-0.043888,0.630485,-0.519142,0.079023,0.064762,0.298179,0.070963,0.533080,3.021579e-15,-0.040382,0.020443,0.229914,0.267896,-0.042185,-0.055186,-0.020176,0.128227,0.144515,0.253504,0.453322,-0.206294,0.050670,-0.079414,0.035165,4.503283e-15,0.106784,0.082911,0.249866,0.211364,0.140251,-0.417351,0.296299,0.036493,-0.347139,...,-0.084218,0.175022,-0.256635,0.183724,-0.734204,0.596193,0.113683,0.047409,0.515204,-0.225078,-0.238918,-0.082427,0.504430,0.229747,0.291917,0.254713,-0.037295,0.155912,-0.438977,-0.564875,0.211184,-0.478583,-0.025655,0.312364,2.604067e-17,0.068224,0.075462,-0.222567,0.117992,0.818927,-0.743018,0.541410,0.058037,-1.001448,0.209712,0.068751,0.543379,-0.440242,-0.491934,0.411806
6599ebbef3d868afac9daa4f80fa075675cf03bc,-0.174390,3.840776e-15,0.198967,0.360072,0.834812,0.378882,-0.739847,-0.770333,0.490844,0.213027,0.053712,0.345802,0.392037,0.652506,-3.232435e-03,-0.589733,1.276236,-0.028276,0.501791,0.255501,-0.602600,-0.498386,1.503724,0.452264,0.427541,0.071556,0.422786,1.085381,-0.608963,0.939713,-1.020146e-14,0.303983,-0.105530,1.121386,1.650522,-0.274607,0.030261,1.321126,-0.483818,-0.632371,...,-0.995895,0.205876,-0.698931,0.218515,-2.429900,0.557458,0.374872,-0.918986,1.516496,2.931786,0.709938,-0.895964,-0.246502,-1.501710,0.777370,-1.303227,0.705868,1.903295,-1.601298,-1.854978,0.249152,0.063434,0.467104,-2.488892,-9.317183e-15,-0.045003,0.395348,-0.884474,-0.476905,1.773109,-2.041111,0.885568,1.045561,-1.263808,1.106392,0.695091,0.648254,-0.332068,-0.477000,0.456160
eb5c7f3ff921ad6469b79cc8a3c122648204ece4,0.077796,-1.102958e-15,-0.153932,0.023555,-0.058868,-0.081289,-0.058218,0.041657,-0.2087

#**Searching**

In [0]:
from gensim.models import Word2Vec, KeyedVectors
Bio_text_vec = KeyedVectors.load_word2vec_format('/content/drive/My Drive/COVID-19/TopicModel/PubMed-and-PMC-w2v.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
search="coronavirus,corona virus,covid-19,2019-ncov,ncov,sars-cov-2,sars,sars-cov,mers,mers-cov,severe acute respiratory syndrome,middle east respiratory syndrome"
all_search_list=search.split(",")
Search_NOT="animal,equine,porcine,calves,dog,canine,feline,bat,camel"
all_Search_NOT_list=Search_NOT.split(",")
Search_AND="incubation period,onset,exposure"
all_Search_AND_list=Search_AND.split(",")


In [0]:

all_most_similar=dict()
for w in all_search_list:
    try:
        s=Bio_text_vec.most_similar(w.lower())
        #search_list.append(w)
        #print(w)
        #print(s)
        most_similar_list=[]
        for i in range(len(s)):
            if s[i][1]>=.5:
                #print(s[i][1])
                most_similar_list.append(s[i])
        all_most_similar[w]=most_similar_list

    except:
        #print("Not")
        pass
print("Research topic",all_most_similar)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Research topic {'coronavirus': [('SARS-CoV', 0.8422620892524719), ('NL63', 0.7770477533340454), ('SARS-coronavirus', 0.764738917350769), ('coronaviruses', 0.7604833245277405), ('TGEV', 0.7564301490783691), ('OC43', 0.7498708963394165), ('229E', 0.7488416433334351), ('HCoV', 0.747186541557312), ('Coronavirus', 0.7404941916465759), ('HCoV-229E', 0.7396430969238281)], 'sars': [('Karelinia', 0.6381598114967346), ('Metapenaeopsis', 0.6173914074897766), ('Ostlu', 0.6147822141647339), ('Sawyeria', 0.6139849424362183), ('tripos', 0.6099517345428467), ('Salmincola', 0.6099454164505005), ('Rhinelepis', 0.6081749200820923), ('Ergenstrema', 0.605999231338501), ('Floriceps', 0.6050954461097717), ('iricolor', 0.604967474937439)], 'mers': [('mer', 0.6761701107025146), ('20-mers', 0.6730425953865051), ('25mer', 0.6383712291717529), ('25-mer', 0.6352108716964722), ('15-mers', 0.6309475898742676), ('10-mers', 0.6302592158317566), ('nt-long', 0.6299068927764893), ('15mer', 0.6254558563232422), ('19-mer',

In [0]:
doc_topic_df = pd.read_csv('/content/drive/My Drive/COVID-19/TopicModel/W_Bio_50.csv',index_col=0)

In [0]:
print(all_most_similar.get(1))

None
